# Module 02: Technical Indicators

## Goal
Implement and visualize key technical indicators: RSI, MACD, and Bollinger Bands.

## Setup

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Try importing TA-Lib, fallback to manual calculation if not present
try:
    import talib
    HAS_TALIB = True
except ImportError:
    HAS_TALIB = False
    print("TA-Lib not found. Using manual pandas calculations.")

## 1. Load Data

In [ ]:
df = pd.read_csv('../demo_data/nifty50_sample.csv',parse_dates=['Date'], index_col='Date')
df.sort_index(inplace=True)
df.head()

## 2. Relative Strength Index (RSI)
RSI measures the speed and change of price movements. RSI > 70 is overbought, < 30 is oversold.

In [ ]:
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

    rs = gain / loss
    return 100 - (100 / (1 + rs))

if HAS_TALIB:
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
else:
    df['RSI'] = calculate_rsi(df['Close'])

## 3. MACD (Moving Average Convergence Divergence)
Trend-following momentum indicator.

In [ ]:
if HAS_TALIB:
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
else:
    # Manual Calculation
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

## 4. Visualization

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05,
                    row_heights=[0.5, 0.25, 0.25],
                    subplot_titles=('Price', 'RSI', 'MACD'))

# Row 1: Candlestick
fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='OHLC'), row=1, col=1)

# Row 2: RSI
fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='purple')), row=2, col=1)
fig.add_hline(y=70, row=2, col=1, line_dash="dash", line_color="red")
fig.add_hline(y=30, row=2, col=1, line_dash="dash", line_color="green")

# Row 3: MACD
fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], name='MACD', line=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MACD_signal'], name='Signal', line=dict(color='orange')), row=3, col=1)
fig.add_trace(go.Bar(x=df.index, y=df['MACD_hist'], name='Histogram'), row=3, col=1)

fig.update_layout(title='Technical Indicators Dashboard', xaxis_rangeslider_visible=False, height=900)
fig.show()